<a href="https://colab.research.google.com/github/bipinupd/notebooks/blob/main/Veo3_Examples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# PIP Installs
import sys

# https://pypi.org/project/moviepy/
!{sys.executable} -m pip install moviepy

In [2]:
!gcloud auth application-default login

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=BrsZMg7rmPnVN2okEcczdYYZitp6kC&prompt=consent&token_usage=remote&access_type=offline&code_challenge=hUtPvCJEs7SHeIzw3MRkfoMldQOJENwLJDWh6B3TEiU&code_challenge_method=S256

Once finished, enter the verification code provided in your browser: 4/0AVGzR1CrTf-NiJhZvp2kxOXzC5F7V6YFiiKDfsZkryHQD5IYvdR8JOZIlL3DcfQ3UosJ9w

Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).
Ca

In [4]:
from google.colab import drive
drive.mount('/content/theme_generation')

Mounted at /content/theme_generation


In [15]:
import google.genai as genai
from google.genai import types
from PIL import Image
import os
PROJECT_ID = "bipin-dev-project-465919"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
LOCATION = "global"
VEO_MODEL_ID = "veo-3.0-fast-generate-001" # @param ['veo-2.0-generate-001', 'veo-3.0-fast-generate-001', 'veo-3.0-generate-001'] {"allow-input":true, isTemplate: true}
output_gcs_uri = "gs://veo3-bucket-example/example" # @param {type: "string"}
resolution = "1080p" # @param ["720p","1080p"]
client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

In [16]:
import pandas as pd
df = pd.read_csv("theme_generation/MyDrive/theme_generation/input/input-data.csv")

In [17]:
df.head()

,shop_id,theme_generation_input,product_id,title,description,full_category
0,65277395053,clothing store,7800069783661,Khaos Picnic Rug,"<p><span>Built for moto families and more, thi...",Home & Garden > Lawn & Garden > Outdoor Living...
1,66008907855,A store offering personalized luggage tags wit...,7833743392847,Custom Laser Engraved Luggage Tag,"<p>Sand out while you travel! Durable, lightwe...",Luggage & Bags > Luggage Accessories > Luggage...
2,66247098479,"PERFUMES, JOYAS Y RELOJES",7867097251951,Khamrah Qawha,<p><img><img><img><img><img></p>,"Food, Beverages & Tobacco > Beverages > Coffee"
3,66248835183,A store specializing in wireless car accessori...,7865957875823,Wireless CarPlays Adapter Lag-Free Connectivit...,"<h3 data-start=""65"" data-end=""91""><strong data...",Vehicles & Parts > Vehicle Parts & Accessories...
4,66394128467,Gameday Apparel,7672919949395,Spotlight,NaN,Arts & Entertainment > Hobbies & Creative Arts...


In [18]:
VEO_PROMPT = """You are an expert Cinematic Prompt Engineer and a creative director for Veo. Your purpose is to transform a user's basic prompt and optional reference image into a masterful, detailed, and technically rich Veo prompt that will guide the model to generate a high-quality video.

Follow this structured process:

**1. Deconstruct & Anchor (Fidelity is Paramount). **
Your primary directive is to preserve the user's vision and the provided visual facts.

*   **Image Fidelity: If a reference image is present, meticulously identify every key visual anchor: subject(s) (whether person, animal, or machine), objects, setting, composition (e.g., high-angle), lighting (e.g., golden hour), and overall aesthetic (e.g., photorealistic, B&W). These elements are non-negotiable. Crucially, you must repeat these core visual anchors throughout your final prompt to reinforce them and prevent the model from drifting. For example, if the image shows a 'red cup on a wooden table,' your prompt should reiterate 'red cup' and 'wooden table' rather than just 'the cup' or 'the table.' The animation you describe must be a plausible and direct evolution of the static scene.
*   **Intent Fidelity: Flawlessly identify and preserve the user's core intent from their text (e.g., 'make it slow motion,' 'animate the dancer'). Your entire augmentation must be a creative elaboration that serves to fulfill this specific goal, never contradicting or ignoring it.
*   **Conceptual Fidelity: If the prompt is abstract (e.g., 'the feeling of nostalgia'), your first task is to translate this concept into a concrete, powerful visual narrative. Brainstorm visual metaphors and scenes that embody the feeling (e.g., for 'nostalgia,' you might describe a 'super 8 film aesthetic showing a sun-faded photograph of a childhood memory').

**2. Build the World (Cinematic & Sensory Enrichment). **
Building upon the anchored foundation, construct an immersive scene by layering in specific, evocative details.

*   **Subject & Action: Add specificity, emotion, and texture.
    *   Instead of "a woman," describe "an elderly woman with kind, crinkled eyes and silver hair pulled into a neat bun." Instead of "dancing," describe "performing a lively 1920s Charleston, feet swiveling and legs kicking, her beaded dress shimmering under the spotlights."
    *   Instead of "a robot," describe "a weathered, bipedal industrial robot with hydraulic arms and a single glowing red optical sensor, its metallic chassis covered in scratches and grime." Instead of "a car moves," describe "a sleek, black electric sports car glides silently down a rain-slicked street, its tires kicking up a fine mist."
    *   Incorporate Diversity: When a subject is generic (e.g., "a person," "a scientist"), actively and thoughtfully incorporate diversity in age, ethnicity, cultural background, ability, and body type to create richer, more representative scenes. Similarly, when a subject is a generic machine (e.g., "a car," "a drone"), incorporate diversity in its design, era, and condition. Instead of a generic car, specify a 'gleaming, futuristic maglev vehicle,' a 'rusty 1970s muscle car,' or a 'utilitarian, armored transport rover.'
    *   Weave in Secondary Motion & Texture: Bring the scene to life by describing subtle environmental interactions ("wisps of her hair flutter in a gentle breeze," "a tiny wisp of steam rises from a porcelain teacup," "sparks fly from a grinding gear," "oil drips slowly from a complex hydraulic joint") and tangible surfaces ("the glistening, undulating mass of the creature," "the rough, weathered bark of an ancient oak tree," "the smooth, polished chrome of a vehicle's exterior reflecting neon lights").

*   **Scene & Ambiance: Build a complete world. Specify the location (a sun-drenched tropical beach, a cluttered artist's studio), time of day (golden hour, twilight), weather (a fine mist, heavy downpour), and background elements. Use descriptive lighting to establish a mood ("soft morning sunlight streams through a window, creating long shadows," "the eerie, pulsating glow of a green neon sign on a rain-slicked street," "volumetric light rays pierce a dense forest canopy").

**3. Direct the Camera (Technical & Stylistic Specification). **
Translate the visual concept into precise, professional filmmaking language. Combine camera, lens, and style terms to create a cohesive directorial vision.

*   **Camera & Movement: Don't just state a shot; combine it with a movement.
    *   Use precise terms: "extreme close-up," "macro shot," "wide shot," "low-angle shot," "bird's-eye view," "dutch angle," "POV shot."
    *   Incorporate dynamic or subtle movements: "slow dolly in," "tracking shot following the subject," "sweeping aerial drone shot," "handheld shaky cam."
    *   Example Combination: A low-angle tracking shot follows the hero.

*   **Lens & Optical Effects: Add a layer of photographic detail.
    *   Specify effects: "shallow depth of field with creamy bokeh," "deep depth of field," "cinematic lens flare," "rack focus," "wide-angle lens," "telephoto lens compression."
    *   Example Combination: An extreme close-up with a slow dolly in, shot with a shallow depth of field to create a beautiful bokeh effect in the background.

*   **Overall Style & Mood: Define the final, cohesive aesthetic with specific keywords.
    *   Be specific and provide multiple descriptors: "Photorealistic, hyperrealistic, 8K, cinematic," or "Ghibli-inspired 2D animation, watercolor style, whimsical," or "Film noir style, deep shadows, stark highlights, black and white," or "1980s vaporwave aesthetic, neon grid, retro-futuristic."

**4. Synthesize & Finalize. **

*   **Final Output: Your final output must be a in a json as given below

{
  "video_style": {
    "format": "4K (UHD)",
    "transitions": "",
    "effects": "Color Grade",
    "aspect_ratio": "16:9"
  },
  "cinematography": {
    "depth_of_field": "Deep",
    "white_balance": "Daylight",
    "tone": "Naturalistic",
    "lighting": "Natural Light",
    "focus": "Autofocus",
    "exposure": "Properly Exposed"
  },
  "scene": {
    "environment": "Rural",
    "time_of_day": "Afternoon",
    "location": "The English Garden, Munich, over the Isar River and the Chinese Tower."
  },
  "subject": {
    "description": "A majestic eagle with brown and white plumage.",
    "wardrobe": "",
    "action": "Soaring gracefully over the landscape, following the river, and then circling a pagoda before flying off."
  },
  "shot": {
    "composition": "Bird's-eye view",
    "type": "Establishing shot",
    "camera_motion": "Tracking shot",
    "frame_rate": "24 fps (Cinematic)",
    "film_grain": "Fine"
  },
  "visual_details": {
    "style": "Cinematic",
    "color_palette": "Cool Tones",
    "mood": "Serene",
    "action": "The eagle's flight path reveals the Chinese Tower in a clearing below.",
    "props": "Chinese Tower (Pagoda)"
  },
  "motion": {
    "camera_movement": "Slow, smooth aerial gliding shot, following the subject and then orbiting a landmark.",
    "subject_movement": "The eagle soars in a calm, unhurried manner, gliding on air currents.",
    "speed": "Real-time",
    "duration": "Long Take (8s)"
  },
  "audio": {
    "ambient": "The gentle sound of wind, distant park ambience.",
    "music": "A subtle, soaring orchestral score.",
    "sound_effects": "The occasional cry of the eagle."
  },
  "dialogue": {
    "character": "",
    "line": "",
    "subtitles": false
  }
}

*   **Self-Correction Checklist: Before finalizing, quickly check your work:
    *   **Anchoring:** Have I repeated the core visual elements from the image/prompt?
    *   **Specificity:** Is 'a bug' described as 'a seven-spotted ladybug with glistening elytra'? Is 'a drone' described as 'a sleek quadcopter drone with carbon fiber arms and a gimbal-mounted 4K camera'?
    *   **Cinematography:** Does the prompt combine shot type, movement, and lens effect?
    *   **Cohesion:** Do all the details (lighting, action, style) serve a single, focused scene?
    *   **Safety:** Ensure the prompt adheres to responsible AI guidelines, avoiding the generation of harmful or prohibited content.

"""

In [ ]:
from google import genai
import google.genai.types as genai_types
from PIL import Image
from io import BytesIO
from typing import Any, Dict, List, Optional
import json
from PIL import Image
import time
from google.cloud import storage
from IPython.display import Video, HTML

theme_ideas: Dict[str, List[Dict[str, Any]]] = {}
i=0
for _, row in df.head().iterrows():
    title = row.get('title', 'Untitled Product')
    category = row.get('full_category', 'Uncategorized')
    title_name = title.lower().replace(" ", "_")
    pil_image = Image.open(f"theme_generation/MyDrive/theme_generation/product_output_folder/{title_name}/0.png")
    video_prompt_response = client.models.generate_content(
            model="gemini-2.5-flash",
            contents=[VEO_PROMPT, pil_image],
            config=genai_types.GenerateContentConfig(
                thinking_config=genai_types.ThinkingConfig(thinking_budget=-1)
            ),
        )
    video_prompt = video_prompt_response.text.strip()
    input_image = genai_types.Image.from_file(location=f"theme_generation/MyDrive/theme_generation/product_output_folder/{title_name}/0.png")
    operation = client.models.generate_videos(
        model=VEO_MODEL_ID,
        prompt=video_prompt,
        image=input_image,
        config=types.GenerateVideosConfig(
            # At the moment the config must not be empty
            resolution=resolution,
            number_of_videos=1, # 1 video generated per request
            output_gcs_uri=output_gcs_uri,
        ),
    )

    # Waiting for the video(s) to be generated
    while not operation.done:
        time.sleep(20)
        operation = client.operations.get(operation)
        print(operation)

    # print(operation.result.generated_videos)

    for n, generated_video in enumerate(operation.result.generated_videos):
        if hasattr(generated_video, 'video') and hasattr(generated_video.video, 'uri'):
            gcs_uri = generated_video.video.uri
            print(f"Attempting to download from GCS URI: {gcs_uri}")

            # Parse bucket and blob name from gcs_uri (e.g., gs://my-bucket/my-folder/video.mp4)
            if gcs_uri.startswith("gs://"):
                bucket_name = gcs_uri.split('/')[2]
                blob_name = '/'.join(gcs_uri.split('/')[3:])
                destination_file_name = f'video_{i}.mp4'
                i=i+1

                try:
                    storage_client = storage.Client(project=PROJECT_ID)
                    bucket = storage_client.bucket(bucket_name)
                    blob = bucket.blob(blob_name)
                    blob.download_to_filename(destination_file_name)
                    print(f"Downloaded video to {destination_file_name}")

                    # Display the video
                    display(Video(destination_file_name, width=640))
                except Exception as e:
                    print(f"Error downloading video from GCS: {e}")
            else:
                print(f"URI format not supported for direct download: {gcs_uri}")
        else:
            print(f"Could not find a downloadable URI in generated_video {n}")





name='projects/bipin-dev-project-465919/locations/global/publishers/google/models/veo-3.0-fast-generate-001/operations/c7a972c1-5ac3-41c6-8521-5d652fb8ad88' metadata=None done=None error=None response=None result=None
name='projects/bipin-dev-project-465919/locations/global/publishers/google/models/veo-3.0-fast-generate-001/operations/c7a972c1-5ac3-41c6-8521-5d652fb8ad88' metadata=None done=None error=None response=None result=None
name='projects/bipin-dev-project-465919/locations/global/publishers/google/models/veo-3.0-fast-generate-001/operations/c7a972c1-5ac3-41c6-8521-5d652fb8ad88' metadata=None done=None error=None response=None result=None
name='projects/bipin-dev-project-465919/locations/global/publishers/google/models/veo-3.0-fast-generate-001/operations/c7a972c1-5ac3-41c6-8521-5d652fb8ad88' metadata=None done=None error=None response=None result=None
name='projects/bipin-dev-project-465919/locations/global/publishers/google/models/veo-3.0-fast-generate-001/operations/c7a972c1-

/usr/local/lib/python3.12/dist-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


Downloaded video to video_0.mp4


name='projects/bipin-dev-project-465919/locations/global/publishers/google/models/veo-3.0-fast-generate-001/operations/77ab339e-e583-4334-a588-0336f15d0d35' metadata=None done=None error=None response=None result=None
name='projects/bipin-dev-project-465919/locations/global/publishers/google/models/veo-3.0-fast-generate-001/operations/77ab339e-e583-4334-a588-0336f15d0d35' metadata=None done=None error=None response=None result=None
name='projects/bipin-dev-project-465919/locations/global/publishers/google/models/veo-3.0-fast-generate-001/operations/77ab339e-e583-4334-a588-0336f15d0d35' metadata=None done=None error=None response=None result=None
name='projects/bipin-dev-project-465919/locations/global/publishers/google/models/veo-3.0-fast-generate-001/operations/77ab339e-e583-4334-a588-0336f15d0d35' metadata=None done=None error=None response=None result=None
name='projects/bipin-dev-project-465919/locations/global/publishers/google/models/veo-3.0-fast-generate-001/operations/77ab339e-

/usr/local/lib/python3.12/dist-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


Downloaded video to video_1.mp4


name='projects/bipin-dev-project-465919/locations/global/publishers/google/models/veo-3.0-fast-generate-001/operations/81bc0044-24ea-48e2-bf58-03c748efc724' metadata=None done=None error=None response=None result=None
